In [2]:
import pandas as pd
import subprocess
import csv

# Load the CSV with gene IDs
df = pd.read_csv("/Users/rishabjain/Desktop/Research/choformer/expression/data/rna/cleaned GSE77800/final_sum_nozero_log.csv")

,gene,expression
0,100760238,0.148916
1,100760532,0.387830
2,100761113,0.384858
3,100760824,0.762337
4,100754742,0.047732
...,...,...
13247,103159063,0.713986
13248,103159064,0.200290
13249,103159065,0.380840
13250,100760682,0.417143


In [ ]:
import concurrent.futures
import subprocess
import csv

# Function to fetch sequence for a given gene_id
def fetch_sequence(gene_id, expression_value):
    print(f"Fetching sequence for {gene_id}")
    sequence = ""
    try:
        # Run efetch command to fetch the sequence from NCBI
        result = subprocess.run(
            ["efetch", "-db", "nuccore", "-id", str(gene_id), "-format", "fasta"],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        
        # If the result has data, extract the sequence
        if result.stdout:
            sequence = result.stdout.strip()  # Remove extra spaces/newlines
        
    except Exception as e:
        print(f"Error fetching sequence for {gene_id}: {e}")
    
    return [gene_id, expression_value, sequence]

# List to store gene sequences and expressions
results = []

# Use ThreadPoolExecutor to run subprocesses in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(fetch_sequence, row.gene, row.expression) for row in df.itertuples()]

    # Collect the results as they complete
    for future in concurrent.futures.as_completed(futures):
        results.append(future.result())

# Save the results to a new CSV
with open("gene_sequences.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write the header
    writer.writerow(["Gene_ID", "Expression", "Sequence"])
    # Write the data
    writer.writerows(results)

print("Gene sequences saved to gene_sequences.csv")